In [1]:
#imports
import metapy
import csv
import requests
import re
from bs4 import BeautifulSoup,Comment
from selenium import webdriver
import time
import json
import nltk
import pandas as pd

In [2]:
#Function for calling soup
def func(url):
    response = requests.get(url)
    html = response.content
    soup=BeautifulSoup(html,'lxml')
    return soup

In [3]:
def uncomment(table):
    text1 = table.findAll(text=lambda x:isinstance(x, Comment))
    #comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comments in text1:
       commentsoup = BeautifulSoup(comments, "lxml")
       blah= commentsoup.find('td')
       if(blah):
        comments.insert_after(commentsoup.find('body').find('td'))
       comments.extract()
    return table

In [4]:
def make_list(table):
    length=len(table.find_all("td",text=True))
    val_list=[]
    i=0
    j=0
    for i in range(0,length,6):
        val=[]
        year = (table.find_all("td",text=True)[i].text)
        #print("1:"+str(i))
        course_name = table.find_all("a",text=True)[j].text.encode('utf-8').strip()
        j+=1
        i+=2
        #print("2:"+str(i))
        term = (table.find_all("td",text=True)[i].text.encode('utf-8').strip())
        i+=1
        #print("3:"+str(i))
        course = (table.find_all("td",text=True)[i].text.encode('utf-8').strip())
        i+=1
        #print("4:"+str(i))
        description = (table.find_all("td",text=True)[i].text.encode('utf-8').strip())
        i+=1
        #print("5:"+str(i))
        #print(course)
        if(course == "IS 590"):
            continue
        val = [year,term,course,course_name,description]
        val_list.append(val)
    return val_list

In [5]:
def write_to_file(val_list,name):
    with open (name,'w',newline='') as file:
        writer=csv.writer(file)
        writer.writerow(["Year","Term","Course_ID","Course_Name","Course_Description"])
        for row in val_list:
            writer.writerow(row)

In [6]:
import os
def merge_csv(list_files):
    try:
        os.remove("final.csv")
    except OSError:
        pass
    fout = open("final.csv","a")
    for line in open(list_files[0]):
        fout.write(line)
    for l in range(1,len(list_files)):
        f = open(list_files[l])
        next(f)
        for l in f:
            fout.write(l)
        f.close()
    fout.close()

In [7]:
list_files = ["engg_file.csv","ischool_file.csv","giesbusiness_file.csv","stats_file.csv","is590ischool_file.csv"]

In [8]:
#For Engineering
url = "https://courses.illinois.edu/search?year=2018&term=fall&keyword=&keywordType=qs&instructor=&collegeCode=KP&subjectCode=&creditHour=&degreeAtt=&courseLevel=&genedCode1=&genedCode2=&genedCode3=&genedType=all&partOfTerm=&_online=on&_open=on&_evenings=on"
soup = func(url)
table = soup.find('table', attrs={'id': 'search-result-dt'})
table = uncomment(table)
val_list = make_list(table)
write_to_file(val_list,"engg_file.csv")

In [ ]:
#For iSchool
url = "https://courses.illinois.edu/search?year=2018&term=fall&keyword=&keywordType=qs&instructor=&collegeCode=LP&subjectCode=&creditHour=&degreeAtt=&courseLevel=&genedCode1=&genedCode2=&genedCode3=&genedType=all&partOfTerm=&_online=on&_open=on&_evenings=on"
soup = func(url)
table = soup.find('table', attrs={'id': 'search-result-dt'})
table = uncomment(table)
val_list = make_list(table)
write_to_file(val_list,"ischool_file.csv")

In [ ]:
#Business
url = "https://courses.illinois.edu/search?year=2018&term=fall&keyword=&keywordType=qs&instructor=&collegeCode=KM&subjectCode=&creditHour=&degreeAtt=&courseLevel=&genedCode1=&genedCode2=&genedCode3=&genedType=all&partOfTerm=&_online=on&_open=on&_evenings=on"
soup = func(url)
table = soup.find('table', attrs={'id': 'search-result-dt'})
table = uncomment(table)
val_list = make_list(table)
write_to_file(val_list,"giesbusiness_file.csv")

In [ ]:
#Stats
url = "https://courses.illinois.edu/search?year=2018&term=fall&keyword=&keywordType=qs&instructor=&collegeCode=KV&subjectCode=STAT&creditHour=&degreeAtt=&courseLevel=&genedCode1=&genedCode2=&genedCode3=&genedType=all&partOfTerm=&_online=on&_open=on&_evenings=on"
soup = func(url)
table = soup.find('table', attrs={'id': 'search-result-dt'})
#print(table)
table = uncomment(table)
val_list = make_list(table)
write_to_file(val_list,"stats_file.csv")

In [ ]:
url = "https://courses.illinois.edu/search/schedule/2018/fall/IS/590?sess="
soup=func(url)
#print(soup.prettify())
table = soup.find('script',attrs={'type' : 'text/javascript'})
jsonval ='[%s]' %(table.text.split('[',1)[1].rsplit(']',1)[0],)
jsonvalue = '{%s}' %(jsonval.split('{',1)[1].rsplit('}',1)[0],) 
value = json.loads(jsonval)
val_list=[]
for i in range(len(value)):
    #regex = re.compile('<.?*>')
    regex = re.compile('<[^<]+?>',re.UNICODE)
    #regex.encode('utf-8').strip()
    section = re.sub(regex, '', value[i]["section"],re.UNICODE)
    #print(value[i]["section"].)
    course_no =  "IS 590" + section
    course_name = value[i]['sectionTitle'].encode('utf-8').strip()
    year ="2018"
    term = "Fall"
    course_info = value[i]["info"].encode('utf-8').strip()
    val = [year,term,course_no,course_name,course_info]
    val_list.append(val)
write_to_file(val_list,"is590ischool_file.csv")
#print(table.text)

In [ ]:
merge_csv(list_files)

In [ ]:
#Checking formats
df = pd.read_csv('is590ischool_file.csv')
df = pd.read_csv('stats_file.csv')
df = pd.read_csv('giesbusiness_file.csv')
df = pd.read_csv('ischool_file.csv')
df = pd.read_csv('engg_file.csv')
df = pd.read_csv('final.csv')